# 10.0 Graphs

Add some common functions for representing graphs using an adjacency list and converting between graphs and python lists.

In [1]:
import collections


class Graph(object):
    """Adjacency list representation of a graph."""
    
    def __init__(self):
        self.vertices = collections.defaultdict(set)

    def add_edge(self, v, w, directed=True):
        self.vertices[v].add(w)
        if not(directed):
            self.add_edge(w, v, directed=True)
        elif w not in self.vertices:
            self.vertices[w] = set()  # Vertex with no out edges.


def make_graph(pairs, directed=True):
    """Return a graph initialized from a python list of vertices."""

    g = Graph()
    for v, w in pairs:
        g.add_edge(v, w, directed=directed)
    return g


def make_list_bfs(graph, root):
    """Return a python list of vertices ordered by bfs traversal."""

    visited, discovered = [], set([root])
    queue = collections.deque([root])

    while len(queue) > 0:
        v = queue.popleft()
        visited.append(v)
        for w in graph.vertices[v]:
            if w not in discovered:
                discovered.add(w)
                queue.append(w)

    return visited


def make_list_dfs(graph, root):
    """Return python list of vertices ordered by dfs traversal."""

    visited, discovered = [root], set([root])
    stack = collections.deque([root])    

    while len(stack) > 0:
        v = stack[0]
        for w in graph.vertices[v]:
            if w not in discovered:
                discovered.add(w)
                stack.appendleft(w)
                # Visit order reflects the order in which vertics are
                # first seen during the depth first traversal eg the
                # descending portion of the recursion.
                visited.append(w)
                break
        # After returning from recursive traversal of subgraph of v,
        # then v again is topmost element and should be popped.
        if stack[0] is v:
            stack.popleft()

    return visited

## 10.1 Determine if a cycle exists

### Problem Statement
Given an undirected graph determine if it contains a cycle.

In [2]:
import collections
import unittest


def has_cycle(graph):
    """Return True when the graph has a cycle."""

    # A cycle exists in an undirected graph when there is a back edge.
    # A back edge connects to a vertex discovered before its' parent.
    # Back edges are identified by performing a dfs traversal, recording
    # the parent of each vertex during the traversal, and looking for
    # an edge from v to w such that w is also the parent of v.
    root = next(iter(graph.vertices.keys()))  # Start from any vertex.
    visited, discovered, processed = [root], set([root]), set()
    stack, parents = collections.deque([root]), {root:root}
    
    while len(stack) > 0:
        v = stack[0]
        for w in graph.vertices[v]:
            if w not in discovered:
                discovered.add(w)
                stack.appendleft(w)  # Descending phase of recursion.
                parents[w] = v
            elif w not in processed:
                # Since we are in the ascending phase of the recursion,
                # the order in which we see the vertices is flipped.
                if w != parents[v]:
                    return True
        if stack[0] is v:  # Ascending phase of recursion.
            processed.add(v)
            stack.popleft()
    
    return False


class HasCycleTest(unittest.TestCase):
    
    def setUp(self):
        # A tree is by definition an acyclic undirected graph.
        self.edges1 = [(1,2),(2,3),(2,4)]
        self.edges1_cycle = False
        # Introduce a cycle into the tree.
        self.edges2 = [(1,2),(2,3),(2,4),(4,3)]
        self.edges2_cycle = True
        # Random acyclic graph.
        self.edges3 = [(1,2),(2,3),(2,4),(4,5),(5,6)]
        self.edges3_cycle = False
        # Introduce a cycle into the previous graph.
        self.edges4 = [(1,2),(2,3),(2,4),(4,5),(5,6),(6,3)]
        self.edges4_cycle = True  

    def test_has_cycle(self):
        case = collections.namedtuple('case', ['input','expected'])
        cases = [
            case(self.edges1, self.edges1_cycle),
            case(self.edges2, self.edges2_cycle),
            case(self.edges3, self.edges3_cycle),
            case(self.edges4, self.edges4_cycle),
        ]
        for c in cases:
            g = make_graph(c.input, directed=False)
            rcv = has_cycle(g)
            self.assertEqual(rcv, c.expected)


unittest.main(HasCycleTest(), argv=[''], verbosity=2, exit=False)

test_has_cycle (__main__.HasCycleTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


## 10.2 Remove edges to create even trees

### Problem Statement
You are given a tree with an even number of nodes.  Remove some of the edges from the tree such that the disconnected subtrees that remain have an even number of nodes.  

Implement a function that returns the maximum number of edges that you can remove while still satisfying this requirement.

In [3]:
import collections
import unittest


def count_descendents(graph, root):
    """Return a dictionary with count of descendents per vertex."""
    
    # Perform dfs and count descendents during ascent from recursion.
    parents, discovered = {root:root}, set([root])
    stack, counts = (collections.deque([root]), 
                     collections.defaultdict(int))
    
    while len(stack) > 0:
        v = stack[0]
        for w in graph.vertices[v]:
            if w not in discovered:
                discovered.add(w)
                parents[w] = v
                counts[v] += 1  # Increment count of descendents.
                stack.appendleft(w)  # Descending.
        if stack[0] is v:  # Ascending.
            if parents[v] is not v:  # Avoid double counting root.
                # Accumulate the count of descendents in parent.
                counts[parents[v]] += counts[v]
            stack.popleft()

    return counts


def remove_edges_even_tree(graph, root):
    """Return max edges to remove and maintain an even tree."""
    
    # Maximum number of edges to remove and maintain an even
    # tree is the count of vertices with odd number of descendents.
    counts = count_descendents(graph, root)
    del counts[root]  # Cannot remove root.
    return len([_ for v in counts.values() if v%2 != 0])


class RemoveEdgesEvenTreeTest(unittest.TestCase):
    
    def setUp(self):
        self.edges1 = [(1,2),(1,3),(3,4),(3,5),(4,6),(4,7),(4,8)]
        self.root1 = 1
        self.edges1_max = 2
        self.edges2 = [(1,2),(1,3),(2,4),(2,5),(2,6),(3,7),(3,8),
                       (7,9),(8,10)]
        self.root2 = 1
        self.edges2_max = 3

    def test_remove_edges_even_tree(self):
        case = collections.namedtuple('case', ['edges','root','expected'])
        cases = [
            case(self.edges1, self.root1, self.edges1_max),
            case(self.edges2, self.root2, self.edges2_max),
        ]
        for c in cases:
            g = make_graph(c.edges, directed=False)
            rcv = remove_edges_even_tree(g, c.root)
            self.assertEqual(rcv, c.expected)


unittest.main(RemoveEdgesEvenTreeTest(), argv=[''], verbosity=2, 
              exit=False)

test_remove_edges_even_tree (__main__.RemoveEdgesEvenTreeTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


## 10.3 Create stepword chain

### Problem Statement
Implement a function that given a pair of words and dictionary of valid words, finds the shortest path of one character transformations between the pair of words. If no path is possible, then return the empty path.

In [4]:
import collections
import unittest


def is_one_char_diff(word1, word2):
    """Return True when the words differ by 1 character."""
    if len(word1) != len(word2):
        return False
    count_diff = 0
    for c1, c2 in zip(word1, word2):
        if c1 != c2:
            count_diff += 1
    return count_diff == 1


def stepword(start, end, valid_words):
    """Find shortest path of 1 character transforms from start to end."""
    
    # Perform bfs traversal to find shortest path in undirected graph.
    # At each step, find all valid transforms and record the parent.
    # Shortest path is found by reverse steps through parents.
    parents = {start:start}
    queue, visited = collections.deque([start]), set([start])
    
    while len(queue) > 0:
        v = queue.popleft()
        for w in valid_words:
            if w not in visited and is_one_char_diff(v, w):
                visited.add(w)
                parents[w] = v
                queue.append(w)

    # Solution holds the reverse order path.
    solution, word = [], end
    while word in parents:
        solution.append(word)
        if word is start:
            break
        word = parents[word]
    solution[:] = solution[::-1]  # Reverse order.

    return solution

    
class StepwordTest(unittest.TestCase):

    def test_stepword(self):
        case = collections.namedtuple('case', ['start','end',
                                               'valid_words',
                                               'expected'])
        cases = [
            case('dog','cat',
                 ['dog','dot','dop','dat','cat'],
                 ['dog','dot','dat','cat']),
            # No valid transforms.
            case('dog','cat',
                 ['dog','dot','tod','mat','cat'],
                 []),
            case('best','rise',
                 ['best','four','ruse','hour','rise','home','fill',
                  'memo','bust','type','also','pack','time','look',
                  'only','rust'],
                 ['best','bust','rust','ruse','rise']),
        ]
        for c in cases:
            rcv = stepword(c.start, c.end, c.valid_words)
            self.assertEqual(rcv, c.expected)


unittest.main(StepwordTest(), argv=[''], verbosity=2, exit=False)

test_stepword (__main__.StepwordTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


## 10.4 Beat Snakes and Ladders

### Problem Statement
[Snakes and Ladders](https://en.wikipedia.org/wiki/Snakes_and_Ladders) is a board game played on a 10x10 grid where the goal of the game is to advance from the start position at grid cell 0 to the stop position at grid cell 100.  Players take turns rolling a 6-sided die and advance the number of cells given by the die.  Players who land on cells labeled as the bottom of a ladder advance to top of the ladder.  Players who land on cells labeled as the head of a snake are sent backward to the tail of the snake.

Find the minimum set of moves required to advance through the board, given a list of positions representing the snakes and ladders on the board.

In [5]:
import collections
import unittest


def minimum_moves(snakes_and_ladders):
    """Find the minimum number of moves to finish Snakes and Ladders."""

    # Perform bfs traversal and record min steps to reach each position.
    steps = {0:0}
    queue = collections.deque([0])

    while len(queue) > 0:
        pos = queue[0]
        for roll in range(6,0,-1):  # Try largest rolls first.
            nextpos, nextsteps = pos+roll, steps[pos]+1
            # Handle landing on snake or ladder.
            if nextpos in snakes_and_ladders['snakes']:
                nextpos = snakes_and_ladders['snakes'][nextpos]
            elif nextpos in snakes_and_ladders['ladders']:
                nextpos = snakes_and_ladders['ladders'][nextpos]
            # Clamp rolls which take us past end of board.
            nextpos = min(nextpos, 100)
            # Terminate traversal for this branch when another path 
            # through board reaches next position in fewer steps.
            if nextpos in steps and nextsteps >= steps[nextpos]:
                continue
            steps[nextpos] = nextsteps  # Min steps to reach nextpos.
            if nextpos < 100:  # At least one more step is required.
                queue.append(nextpos)
        if queue[0] is pos:
            queue.popleft()

    return steps[100]


class MinimumMovesTest(unittest.TestCase):

    def setUp(self):
        self.snakes_and_ladders = {
            'snakes': {17:13,52:29,57:40,62:22,88:18,95:51,97:79},
            'ladders': {3:21,8:30,28:84,58:77,75:86,80:100,90:91},
        }
        self.minimum_moves = 6

    def test_minimum_moves(self):
        case = collections.namedtuple('case', ['input','expected'])
        cases = [
            case(self.snakes_and_ladders, self.minimum_moves),
        ]
        for c in cases:
            rcv = minimum_moves(c.input)
            self.assertEqual(rcv, c.expected)


unittest.main(MinimumMovesTest(), argv=[''], verbosity=2, exit=False)

test_minimum_moves (__main__.MinimumMovesTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


## 10.5 Topological sort

### Problem Statement
Given a list of courses each with its' own list of prerequisites, return the order in which courses must be completed in order to satisfy the curriculum.

In [6]:
import collections
import unittest


def dfs(graph, root, visited, order):
    """Perform a dfs labeling vertices with their topological order."""
    
    stack = collections.deque([root])

    while len(stack) > 0:
        v = stack[0]
        for w in graph.vertices[v]:
            if w not in visited:
                visited[v] = -1
                stack.appendleft(w)
        if stack[0] is v:
            visited[v] = order  # Visited maps vertex to its' order.
            order -= 1
            stack.popleft()
    
    return order


def completion_order(courses):
    """Return a valid order in which courses must be completed."""

    # Use the course requirements to build a directed acyclic graph.
    g = Graph()
    for course, prereqs in courses.items():
        for prereq in prereqs:
            g.add_edge(prereq, course, directed=True)

    # Perform repeated dfs over graph until all vertices are visited.
    visited, order = {}, len(g.vertices)
    for v in g.vertices.keys():
        if v not in visited:
            # Decrement order during traversal.
            order = dfs(g, v, visited, order)

    # Build an inverted index from order to course.
    course_order = {seq: course for course, seq in visited.items()}
    courses = [course_order[seq] for seq in sorted(course_order.keys())]
    
    return courses


class CompletionOrderTest(unittest.TestCase):
    
    def setUp(self):
        self.courses1 = {
            '100': [],
            '110': ['100'],
            '120': ['100'],
            '200': ['110','120'],
            '220': ['120'],
            '300': ['200'],
            '350': ['220'],
            '400': ['300','350'],
        }
        self.courses1_order = ['100','110','120','200','300',
                               '220','350','400']

    def test_completion_order(self):
        case = collections.namedtuple('case', ['input','expected'])
        cases = [
            case(self.courses1, self.courses1_order),
        ]
        for c in cases:
            rcv = completion_order(c.input)
            completed = set()
            for course in rcv:
                # Recursively check prerequisites.
                queue = collections.deque(c.input[course])
                while len(queue) > 0:
                    prereq = queue.popleft()
                    self.assertTrue(prereq in completed)
                    queue.extend(c.input[prereq])
                completed.add(course)


unittest.main(CompletionOrderTest(), argv=[''], verbosity=2, exit=False)

test_completion_order (__main__.CompletionOrderTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.005s

OK
